In [67]:
FILE_PATH = ''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        FILE_PATH = os.path.join(dirname, filename)

In [68]:
import tensorflow as tf

# Check if TensorFlow is using GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [69]:
# Method 2: Using open and close explicitly
file = open(FILE_PATH, 'r')
content = file.read()
file.close()

In [70]:
#Remove punctuations
import re
_content = re.sub(r'[^\w\s]', '', content)
content_clean = _content.lower()[:500000]

In [72]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [73]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([content_clean])
unique_cnt = len(tokenizer.word_index)
print("Total unique words in the document is ", unique_cnt)

Total unique words in the document is  7004


In [74]:
total_sentences = content_clean.split('\n')
print("Total sentences are ",len(total_sentences))

Total sentences are  2204


In [75]:
input_sequences = []
for sentence in total_sentences:
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1,len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])

In [76]:
max_len = max([len(x) for x in input_sequences])
print("Size of the longest sentence is ",max_len)

Size of the longest sentence is  351


In [77]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [78]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,    1,  183],
       [   0,    0,    0, ...,    1,  183,   99],
       [   0,    0,    0, ...,    0,  251,  381],
       ...,
       [   0,    0,    0, ...,  920, 1139, 7003],
       [   0,    0,    0, ..., 1139, 7003,   46],
       [   0,    0,    0, ..., 7003,   46, 7004]], dtype=int32)

In [79]:
X = padded_input_sequences[:,:-1]

In [80]:
X.shape

(95615, 350)

In [81]:
y = padded_input_sequences[:,-1]

In [82]:
y.shape

(95615,)

In [83]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=unique_cnt+1)

In [84]:
y.shape

(95615, 7005)

In [85]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [86]:
X.shape

(95615, 350)

In [87]:
y.shape

(95615, 7005)

In [88]:
model_two = Sequential()
model_two.add(Embedding(unique_cnt+1, 128, input_shape=(max_len,)))
model_two.add(LSTM(256))
model_two.add(Dense(unique_cnt+1, activation='softmax'))

In [89]:
model_two.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [90]:
model_two.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 351, 128)       │       896,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7005)           │     1,800,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,091,165 (11.79 MB)

 Trainable params: 3,091,165 (11.79 MB)

 Non-trainable params: 0 (0.00 B)

In [91]:
model_two.fit(X,y,epochs=9)

Epoch 1/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 75s 25ms/step - accuracy: 0.0840 - loss: 6.3668
Epoch 2/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 74s 25ms/step - accuracy: 0.1404 - loss: 5.2581
Epoch 3/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 73s 25ms/step - accuracy: 0.1671 - loss: 4.7788
Epoch 4/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 73s 24ms/step - accuracy: 0.1922 - loss: 4.3333
Epoch 5/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.2259 - loss: 3.8748
Epoch 6/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.2772 - loss: 3.4566
Epoch 7/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.3345 - loss: 3.0757
Epoch 8/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.3921 - loss: 2.7409
Epoch 9/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.4500 - loss: 2.4402


In [92]:
model_two.fit(X,y,epochs=9)

Epoch 1/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.5067 - loss: 2.1644
Epoch 2/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.5561 - loss: 1.9318
Epoch 3/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.6066 - loss: 1.7025
Epoch 4/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.6510 - loss: 1.5172
Epoch 5/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.6871 - loss: 1.3605
Epoch 6/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.7217 - loss: 1.2123
Epoch 7/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.7496 - loss: 1.0973
Epoch 8/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.7763 - loss: 0.9770
Epoch 9/9
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.7989 - loss: 0.8778


In [93]:
model_two.fit(X,y,epochs=5)

Epoch 1/5
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.8190 - loss: 0.7910
Epoch 2/5
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.8386 - loss: 0.7173
Epoch 3/5
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.8527 - loss: 0.6587
Epoch 4/5
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.8646 - loss: 0.6024
Epoch 5/5
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.8765 - loss: 0.5516


In [97]:
model_two.fit(X,y,epochs=5)

Epoch 1/5
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.8865 - loss: 0.5078
Epoch 2/5
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.8912 - loss: 0.4843
Epoch 3/5
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.9012 - loss: 0.4450
Epoch 4/5
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.9037 - loss: 0.4282
Epoch 5/5
2988/2988 ━━━━━━━━━━━━━━━━━━━━ 72s 24ms/step - accuracy: 0.9081 - loss: 0.4020


In [103]:
import time
import numpy as np
# text = "what is the fee"
texts = ['Hello I am','I dont like it here because','I hate','Trees give us','Chicken is very']
for text in texts:
    text = re.sub(r'[^\w\s]', '', text).lower()
    token_text = tokenizer.texts_to_sequences([text])[0]
    padded_token_text = pad_sequences([token_text], maxlen=max_len, padding='pre')
    pos = np.argmax(model_two.predict(padded_token_text))
    for word,idx in tokenizer.word_index.items():
        if idx == pos:
            text = text + " " + word
            print(text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
hello i am speaking
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
i dont like it here because he
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
i hate people
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
trees give us a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
chicken is very old


In [ ]:
    for word,index in tokenizer.word_index.items():
        if index == pos:
            text = text + " " + word
            print(text)
            time.sleep(2)

In [95]:
def predict(text):
    token_text = tokenizer.texts_to_sequences([text])[0]
    padded_token_text = pad_sequences([token_text], maxlen=max_len, padding='pre')
    pos = np.argmax(model_two.predict(padded_token_text))
    for word,index in tokenizer.word_index.items():
        if index == pos:
            text = text + " " + word
            print(text)

In [96]:
predict("Gambhir is very angry because")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Gambhir is very angry because i


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
MS is very silly
